In [1]:
!pip install transformers datasets --quiet

In [2]:
from datasets import Dataset
import pandas as pd
import torch

In [3]:
with open("miTexto.txt", "r", encoding="utf-8") as f:
    lines = f.readlines()

In [4]:
lines = [line.strip() for line in lines if line.strip()]
#lines = lines[:500]  # ← limitar para evitar sobrecarga

df = pd.DataFrame({"text": lines})
dataset = Dataset.from_pandas(df)

In [5]:
from transformers import GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

def tokenize(example):
    return tokenizer(example["text"], truncation=True, padding="max_length", max_length=128)

tokenized_dataset = dataset.map(tokenize, batched=True, remove_columns=["text"])

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Map:   0%|          | 0/43 [00:00<?, ? examples/s]

In [6]:
from transformers import (
    GPT2LMHeadModel,
    Trainer,
    TrainingArguments,
    DataCollatorForLanguageModeling
)

In [7]:
model = GPT2LMHeadModel.from_pretrained("gpt2")
model.resize_token_embeddings(len(tokenizer))


Embedding(50257, 768)

In [8]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [16]:
training_args = TrainingArguments(
    output_dir="./gpt2-discard",
    per_device_train_batch_size=4,
    num_train_epochs=25,
    logging_steps=30,
    report_to="none",
    save_strategy="no",  # ← no guarda nada
    fp16=torch.cuda.is_available()
)

In [17]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer
)

trainer.train()


<ipython-input-17-2113583967>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
30,2.344200
60,1.905100
90,1.557800
120,1.334100
150,1.114500
180,0.940100
210,0.800000
240,0.685600
270,0.657100


TrainOutput(global_step=275, training_loss=1.2497075713764538, metrics={'train_runtime': 30.2259, 'train_samples_per_second': 35.565, 'train_steps_per_second': 9.098, 'total_flos': 70222233600000.0, 'train_loss': 1.2497075713764538, 'epoch': 25.0})

In [18]:
model.save_pretrained("./gpt2-mi-modelin")
tokenizer.save_pretrained("./gpt2-mi-modelin")

('./gpt2-mi-modelin/tokenizer_config.json',
 './gpt2-mi-modelin/special_tokens_map.json',
 './gpt2-mi-modelin/vocab.json',
 './gpt2-mi-modelin/merges.txt',
 './gpt2-mi-modelin/added_tokens.json')

In [19]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, pipeline

model = GPT2LMHeadModel.from_pretrained("./gpt2-mi-modelin")
tokenizer = GPT2Tokenizer.from_pretrained("./gpt2-mi-modelin")

In [26]:
from transformers import pipeline

generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

# generator = pipeline(
#     "text-generation",
#     model="datificate/gpt2-small-spanish",
#     tokenizer="datificate/gpt2-small-spanish"
# )

Device set to use cuda:0


In [27]:
prompt = "En la caverna "
output = generator(prompt, max_new_tokens=200, temperature=0.7, top_k=50)
print(output[0]["generated_text"])

En la caverna iba a continuación o a la luz del fuego, ante todo, unos hombres que puede unos otros objetos, y que, con respecto al sol, nosos reflejadas en las aguas en ello. En cuanto a la luz del fuego y a la luz del mirar y a la luz del fuesen y a la luz mismo, que se encuentra el poder del. entre ellos. Cuando éstos muestra., ilustrada acuerdo, haber cambiado. Cuando éstos muestra sintiera, ha vuelto el cuello y mirar a la luz del fuesen y a la luz mirar a la luz mismo, sino es natural. Y después de esto es éstos mirar a la l
